Código abaixo funcionando bem para destacar as inconsistências e correções

In [ ]:
from bs4 import BeautifulSoup

# Leitura do arquivo HTML
with open("temp11_redacoes_completas.html", "r", encoding="utf-8") as f:
    soup = BeautifulSoup(f, "html.parser")

# Função para aplicar destaque padronizado
def destacar_erros(span):
    # Caso 1: style com cor
    style = span.get("style", "")
    if "color:red" in style:
        span["style"] = "background-color: #fdd; color: red; font-weight: bold;"
    elif "color:#00b050" in style.lower():
        span["style"] = "background-color: #dfd; color: green; font-weight: bold;"

    # Caso 2: classes
    if span.get("class") and "erro" in span["class"]:
        span["style"] = "background-color: #fdd; color: red; font-weight: bold;"
    elif span.get("class") and "certo" in span["class"]:
        span["style"] = "background-color: #dfd; color: green; font-weight: bold;"

# Aplicar formatação a todos os spans
for span in soup.find_all("span"):
    destacar_erros(span)

# Exportar resultado
with open("redacoes_destacadas.html", "w", encoding="utf-8") as f:
    f.write(str(soup))


In [ ]:
from bs4 import BeautifulSoup
import re

ARQ_ORIGEM  = "redacoes_destacadas.html"
ARQ_DESTINO = "redacoes_sem_correcao.html"

# --- 1. carregar HTML --------------------------------------------------------
with open(ARQ_ORIGEM, "r", encoding="utf-8") as f:
    soup = BeautifulSoup(f, "html.parser")

# --- 2. regra de identificação das correções verdes --------------------------
verde_hex   = re.compile(r"#00b050", re.I)
verde_rgb   = re.compile(r"rgb\s*\(\s*0\s*,\s*176\s*,\s*80\s*\)", re.I)  # opcional
verde_pal   = re.compile(r"\bcolor\s*:\s*green", re.I)
verde_bg    = re.compile(r"background-color\s*:\s*#dfd", re.I)           # após highlight

def eh_correção(tag):
    if tag.name != "span":
        return False

    # a) span class="certo"
    classes = tag.get("class", [])
    if any(cls.lower() == "certo" for cls in classes):
        return True

    # b) span verde (mesmo que tenha perdido a classe)
    style = tag.get("style", "")
    if verde_hex.search(style) or verde_rgb.search(style) or verde_pal.search(style) or verde_bg.search(style):
        return True

    return False

# --- 3. remover todas as correções ------------------------------------------
for span in soup.find_all(eh_correção):
    span.decompose()        # remove o elemento e TODO o texto dentro dele

# --- 4. salvar resultado -----------------------------------------------------
with open(ARQ_DESTINO, "w", encoding="utf-8") as f:
    f.write(str(soup))

print(f"Arquivo limpo gerado em: {ARQ_DESTINO}")


Arquivo limpo gerado em: redacoes_sem_correcao.html
